In [1]:
import cv2
import os,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

2024-11-11 21:54:46.252956: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-11 21:54:46.479331: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-11 21:54:46.692426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731342286.869525    1286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731342286.936988    1286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 21:54:47.362408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
voc_root = 'VOCdevkit/VOC2012/'
image_dir = os.path.join(voc_root, 'JPEGImages')
image_files = os.listdir(image_dir)

In [3]:
def selectivesearch(image):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    
    ss.setBaseImage(image)
    
    # Switch to fast but low recall mode
    ss.switchToSelectiveSearchFast()
    
    # Get the proposed regions
    rects = ss.process()
    
    return rects


In [4]:
image_path = '/home/sandy33/Projects/RCNN-From-Scratch/VOCdevkit/VOC2012/JPEGImages/2007_000063.jpg'  # Adjust this path
image = cv2.imread(image_path)

# Perform Selective Search
rects = selectivesearch(image)

# Draw the bounding boxes on the image
for (x, y, w, h) in rects[:100]:  # Limit to the first 100 proposals for clarity
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow("Selective Search Results", image)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [6]:
from typing import Dict

def get_iou(bb1: Dict[str, float], bb2: Dict[str, float]) -> float:
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters:
        bb1 (dict): Dictionary with keys 'x1', 'y1', 'x2', 'y2' representing the first bounding box.
        bb2 (dict): Dictionary with keys 'x1', 'y1', 'x2', 'y2' representing the second bounding box.

    Returns:
        float: IoU value between 0 and 1, inclusive.
    """
    # Assure proper bounding box dimensions.
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
  
    # Calculate coordinates of intersection.
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
  
    # Return 0 if there is no overlap.
    if x_right < x_left or y_bottom < y_top:
        return 0.0
  
    # Calculate intersection area.
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
  
    # Calculate individual areas.
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
  
    # Compute IoU.
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
  
    # Ensure IoU is within the valid range.
    assert 0.0 <= iou <= 1.0
    return iou
